In [16]:
from dotenv import load_dotenv
from openai import AsyncOpenAI, OpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput, WebSearchTool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict, List, Optional
from pydantic import BaseModel
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio
from scraper import fetch_website_contents, fetch_website_links
from IPython.core.display import Markdown
import gradio as gr
import requests

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [8]:
google_api_key = os.getenv('GOOGLE_API_KEY')
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.5-flash", openai_client=gemini_client)

In [ ]:
name = "Infotrack"

with open(f"profiles/{name}.md", "r", encoding="utf-8") as f:
    summary = f.read()

In [ ]:

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [22]:
instruction = f"""You are {name}, the virtual representative of {name}. Your role is to assist visitors on the {name} website by answering questions about the company's information, services, and career opportunities. 
Respond professionally and engagingly, as if speaking to a potential client or future employee. Use the provided company summary to ensure accurate and authentic answers.
If you don't know the answer, politely acknowledge it instead of guessing.
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated. \
If the user is engaging in carrer, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. """

instruction += f"\n\nSummary:\n<Summary>{summary}</Summary>\n\n"
instruction += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
@function_tool
def record_user_details(email, name="Name not provided", notes="not provided"):
    """
    Log questions that the agent cannot answer for future improvement.
    
    Use this when a user asks something you don't have information about or can't answer.
    This helps track knowledge gaps and improve the agent's capabilities over time.
    
    Args:
        question: The exact question or topic the user asked about that you couldn't answer
    
    Returns:
        dict: Confirmation that the question was recorded
    """
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

@function_tool
def record_unknown_question(question):
    """
    Record user contact information and interest details for follow-up.
    
    Use this when a user expresses interest in your carrer/product/service or wants to be contacted.
    Call this function when the user provides their email address or asks to be contacted.
    
    Args:
        email: User's email address (required)
        name: User's full name (optional, defaults to "Name not provided")
        notes: Additional context about their interest, questions, or requirements (optional)
    
    Returns:
        dict: Confirmation that the details were recorded
    """
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

tools = [record_user_details, record_unknown_question]

cbtw_chat_agent = Agent(
        name="CBTW Chat Agent",
        instructions=instruction,
        model=gemini_model,
        tools=tools
)

# Store conversation history
conversation_history = []
async def cbtw_chat(message, history):
    conversation_history.append({
        "role": "user",
        "content": message
    })
    with trace("CBTW Chat Agent"):
        result = await Runner.run(cbtw_chat_agent, conversation_history)
        response = result.final_output
        # Add assistant response to history
        conversation_history.append({
            "role": "assistant",
            "content": response
        })
        return response

In [ ]:
gr.ChatInterface(cbtw_chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Push: Recording interest from Name not provided with email anh.dang.hoang@positivethinking.tech and notes User is interested in career opportunities
Push: Recording Who is your favorite musician? asked that I couldn't answer
